In [1]:
# ! pip install pydantic-ai

## Tools WITHOUT Context 

In [2]:
import random

from pydantic_ai import RunContext, Tool


def roll_dice(name: str = "Rolling dice") -> str:
    """Roll a six-sided dice and return the result."""
    return str(random.randint(1, 6))


rool_dice_tool = Tool(roll_dice)

In [3]:
rool_dice_tool.function()

'5'

In [4]:
print(rool_dice_tool.name)
print(rool_dice_tool.description)

roll_dice
Roll a six-sided dice and return the result.


In [5]:
rool_dice_tool._parameters_json_schema

{'properties': {'name': {'title': 'Name', 'type': 'string'}},
 'required': ['name'],
 'type': 'object',
 'additionalProperties': False}

In [6]:
from typing import Optional


def foobar(a: int, b: str, c: Optional[dict[str, list[float]]] = None) -> str:
    """Get me foobar.

    Args:
        a: apple pie
        b: banana cake
        c: carrot smoothie
    """
    return f"{a} {b} {c}"


foobar_tool = Tool(foobar)

print(foobar_tool.name)
print(foobar_tool.description)

foobar
Get me foobar.


In [7]:
foobar_tool._parameters_json_schema

{'properties': {'a': {'description': 'apple pie',
   'title': 'A',
   'type': 'integer'},
  'b': {'description': 'banana cake', 'title': 'B', 'type': 'string'},
  'c': {'anyOf': [{'additionalProperties': {'items': {'type': 'number'},
      'type': 'array'},
     'type': 'object'},
    {'type': 'null'}],
   'description': 'carrot smoothie',
   'title': 'C'}},
 'required': ['a', 'b', 'c'],
 'type': 'object',
 'additionalProperties': False}

## Tools WITH Context (simple string)

In [8]:
def get_player_name(ctx: RunContext[str]) -> str:
    """Get the player's name."""
    return ctx.deps


get_player_name_tool = Tool(get_player_name, takes_ctx=True)

In [9]:
# ! pip install nest_asyncio

In [10]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
from pydantic_ai import Agent

agent_a = Agent(
    "openai:gpt-4o",
    deps_type=str,
    tools=[get_player_name_tool],
)

In [12]:
result = agent_a.run_sync("Get players name", deps="Anne")
result

RunResult(_all_messages=[ModelRequest(parts=[UserPromptPart(content='Get players name', timestamp=datetime.datetime(2024, 12, 17, 15, 1, 58, 163148, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[ToolCallPart(tool_name='get_player_name', args=ArgsJson(args_json='{}'), tool_call_id='call_410yrIwckDRCr9wSF0GXy8eL', part_kind='tool-call')], timestamp=datetime.datetime(2024, 12, 17, 15, 1, 58, tzinfo=datetime.timezone.utc), kind='response'), ModelRequest(parts=[ToolReturnPart(tool_name='get_player_name', content='Anne', tool_call_id='call_410yrIwckDRCr9wSF0GXy8eL', timestamp=datetime.datetime(2024, 12, 17, 15, 1, 59, 95607, tzinfo=datetime.timezone.utc), part_kind='tool-return')], kind='request'), ModelResponse(parts=[TextPart(content="The player's name is Anne.", part_kind='text')], timestamp=datetime.datetime(2024, 12, 17, 15, 1, 59, tzinfo=datetime.timezone.utc), kind='response')], _new_message_index=0, data="The player's name is Anne.", 

In [13]:
result.all_messages()

[ModelRequest(parts=[UserPromptPart(content='Get players name', timestamp=datetime.datetime(2024, 12, 17, 15, 1, 58, 163148, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='get_player_name', args=ArgsJson(args_json='{}'), tool_call_id='call_410yrIwckDRCr9wSF0GXy8eL', part_kind='tool-call')], timestamp=datetime.datetime(2024, 12, 17, 15, 1, 58, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='get_player_name', content='Anne', tool_call_id='call_410yrIwckDRCr9wSF0GXy8eL', timestamp=datetime.datetime(2024, 12, 17, 15, 1, 59, 95607, tzinfo=datetime.timezone.utc), part_kind='tool-return')], kind='request'),
 ModelResponse(parts=[TextPart(content="The player's name is Anne.", part_kind='text')], timestamp=datetime.datetime(2024, 12, 17, 15, 1, 59, tzinfo=datetime.timezone.utc), kind='response')]

## Tools WITH Context (pydantic)

In [14]:
from pydantic import BaseModel


class Player(BaseModel):
    name: str
    age: int


def get_player(ctx: RunContext[Player], additional_info: Optional[str] = None) -> str:
    """Get the player's name.

    Args:
        additional_info: Additional information which can be used.
    """
    return f"Name: {ctx.deps.name}, Age: {ctx.deps.age}, Additional info: {additional_info}"


get_player_tool = Tool(get_player, takes_ctx=True)

In [15]:
print(get_player_tool.name)
print(get_player_tool.description)
print(get_player_tool._parameters_json_schema)

get_player
Get the player's name.
{'properties': {'additional_info': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'description': 'Additional information which can be used.', 'title': 'Additional Info'}}, 'required': ['additional_info'], 'type': 'object', 'additionalProperties': False}


In [16]:
agent_b = Agent(
    "openai:gpt-4o",
    deps_type=Player,
    tools=[get_player_tool],
)

player = Player(name="Luka", age=25)
result_b = agent_b.run_sync("Get player", deps=player)
result_b.all_messages()

[ModelRequest(parts=[UserPromptPart(content='Get player', timestamp=datetime.datetime(2024, 12, 17, 15, 1, 59, 708795, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[TextPart(content="Could you please provide any additional information or context regarding the player you're interested in? This will help me deliver the most relevant details for you.", part_kind='text')], timestamp=datetime.datetime(2024, 12, 17, 15, 2, tzinfo=datetime.timezone.utc), kind='response')]

## Dependency injection

In [17]:
# Initial Source: https://github.com/airtai/fastagency/blob/main/fastagency/api/dependency_injection.py

from functools import wraps
from inspect import signature
from typing import Any, Callable


def inject_params(f: Callable[..., Any], ctx: Any) -> Callable[..., Any]:

    @wraps(f)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        kwargs.pop("ctx", None)
        return f(**kwargs, ctx=ctx)

    sig = signature(f)
    new_params = [param for name, param in sig.parameters.items() if name != "ctx"]

    wrapper.__signature__ = sig.replace(parameters=new_params)  # type: ignore[attr-defined]

    return wrapper

In [18]:
ctx = RunContext(
    deps=Player(name="Luka", age=25),
    retry=get_player_tool.max_retries,  # Not sure
    messages=[],  # Not sure
    tool_name=get_player_tool.name,
)
f_injected = inject_params(f=get_player_tool.function, ctx=ctx)

In [19]:
f_injected.__signature__

<Signature (additional_info: Optional[str] = None) -> str>

In [20]:
f_injected(additional_info="Hello")

'Name: Luka, Age: 25, Additional info: Hello'

## AG2 Integration

In [21]:
import os

from autogen import AssistantAgent, UserProxyAgent

In [22]:
config_list = [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
)

chatbot = AssistantAgent(
    name="chatbot",
    llm_config={"config_list": config_list},
)

In [23]:
tool = get_player_tool

f_injected_ctx = inject_params(f=tool.function, ctx=ctx)

In [24]:
user_proxy.register_for_execution(name=tool.name)(f_injected_ctx)
chatbot.register_for_llm(description=tool.description, name=tool.name)(f_injected_ctx)

<function __main__.get_player(additional_info: Optional[str] = None) -> str>

In [25]:
chatbot.llm_config["tools"]

[{'type': 'function',
  'function': {'description': "Get the player's name.",
   'name': 'get_player',
   'parameters': {'type': 'object',
    'properties': {'additional_info': {'anyOf': [{'type': 'string'},
       {'type': 'null'}],
      'default': None,
      'description': 'additional_info'}},
    'required': []}}}]

In [26]:
user_proxy.initiate_chat(
    recipient=chatbot, message="Get player, for additional information use 'goal keeper'", max_turns=3
)

User (to chatbot):

Get player, for additional information use 'goal keeper'

--------------------------------------------------------------------------------
chatbot (to User):

***** Suggested tool call (call_S1wA3FAaTLFAt5aLv19xwOXJ): get_player *****
Arguments: 
{"additional_info":"goal keeper"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_player...
User (to chatbot):

***** Response from calling tool (call_S1wA3FAaTLFAt5aLv19xwOXJ) *****
Name: Luka, Age: 25, Additional info: goal keeper
**********************************************************************

--------------------------------------------------------------------------------
chatbot (to User):

The player's name is Luka, aged 25, and is a goal keeper. TERMINATE

--------------------------------------------------------------------------------
User (to chatbot):



------------

ChatResult(chat_id=None, chat_history=[{'content': "Get player, for additional information use 'goal keeper'", 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': 'call_S1wA3FAaTLFAt5aLv19xwOXJ', 'function': {'arguments': '{"additional_info":"goal keeper"}', 'name': 'get_player'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': 'Name: Luka, Age: 25, Additional info: goal keeper', 'tool_responses': [{'tool_call_id': 'call_S1wA3FAaTLFAt5aLv19xwOXJ', 'role': 'tool', 'content': 'Name: Luka, Age: 25, Additional info: goal keeper'}], 'role': 'tool', 'name': 'User'}, {'content': "The player's name is Luka, aged 25, and is a goal keeper. TERMINATE", 'role': 'user', 'name': 'chatbot'}, {'content': '', 'role': 'assistant', 'name': 'User'}, {'content': "It seems like there was no follow-up request. If there's anything else you need, feel free to ask! TERMINATE", 'role': 'user', 'name': 'chatbot'}], summary="It seems like there was no follow-up request. If there's